# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770872230704                   -0.53    8.0
  2   -2.772140531007       -2.90       -1.31    1.0
  3   -2.772170548262       -4.52       -2.53    2.0
  4   -2.772170722197       -6.76       -4.18    2.0
┌ Warning: Negative ρ detected
│   min_ρ = -9.580435397131086e-18
└ @ DFTK /home/runner/work/DFTK.jl/DFTK.jl/src/densities.jl:5
  5   -2.772170722964       -9.12       -4.69    2.0
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770609965739                   -0.53    8.0
  2   -2.772047237126       -2.84       -1.29    1.0
  3   -2.772083337296       -4.44       -2.78    2.0
  4   -2.772083416995       -7.10       -4.05    2.0
  5   -2.772083417798       -9.10       -4.83    2.0


1.7734104695126922

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770777288799                   -0.53    8.0
  2   -2.772056207803       -2.89       -1.31    1.0
  3   -2.772083202408       -4.57       -2.52    2.0
  4   -2.772083417289       -6.67       -4.26    2.0
  5   -2.772083417793       -9.30       -4.89    2.0

Polarizability via ForwardDiff:       1.7725702874478015
Polarizability via finite difference: 1.7734104695126922
